In [32]:
#export
import k1lib, os, dill, time, inspect, json as _json, base64; k1 = k1lib
from typing import List
import k1lib.cli as cli; from k1lib.cli import *
from collections import defaultdict
pygments = k1.dep("pygments")
try: import PIL.Image, PIL; hasPIL = True
except: PIL = k1.dep("PIL"); hasPIL = False
__all__ = ["tag_serve",
           "FromNotebook", "FromPythonFile", "BuildPythonFile", "BuildDashFile", "StartServer", "GenerateHtml", "commonCbs", "serve",
           "baseType", "text", "slider", "html", "json", "date", "serialized", "apiKey", "analyze", "webToPy", "pyToWeb"]

In [33]:
#export
basePath = os.path.dirname(inspect.getabsfile(k1lib)) + os.sep + "serve" + os.sep
def pretty_py(code_string):
    s = pygments.highlight(code_string, pygments.lexers.PythonLexer(), pygments.formatters.HtmlFormatter()).replace('<pre', '<pre class="pre"')
    css = pygments.formatters.HtmlFormatter().get_style_defs('.highlight').replace(".highlight { background: #f8f8f8; }", "")
    return f'<style>{css}</style>{s}'
def pretty_js(code_string):
    from pygments.lexers.javascript import JavascriptLexer
    s = pygments.highlight(code_string, JavascriptLexer(), pygments.formatters.HtmlFormatter()).replace('<pre', '<pre class="pre"')
    css = pygments.formatters.HtmlFormatter().get_style_defs('.highlight').replace(".highlight { background: #f8f8f8; }", "")
    return f'<style>{css}</style>{s}'
def execPy(s:str, node:str, condaEnv:str=None, condaPath:str="~/miniconda3") -> "list[str]":
    fn = s | cli.file(); None | cli.cmd(f"scp {fn} {node}:{fn}") | cli.deref()
    return None | cli.cmd(f"ssh {node} 'source {condaPath}/bin/activate {condaEnv} && python {fn}'") | cli.filt("x") | cli.deref()
def tag_serve(node:str=None, condaEnv:str=None, condaPath:str="~/miniconda3"):
    """Tag that marks the cell that will be extracted to an independent
file and executed. Example::

    # serve(node="user@hostname", condaEnv="torch")

If a remote node is specified, internally, this will run commands on that node using
ssh, so make sure it's reachable via "ssh user@hostname" with default ssh identity,
or else it wouldn't work.

If .condaEnv is specified, then will activate conda before executing the script.
The activation command will be "{condaPath}/bin/activate {condaEnv}"

:param node: what node should the script be executed?
:param condaEnv: if specified, will activate that conda env before executing"""
    return {"node": node, "condaEnv": condaEnv}
class FromNotebook(k1.Callback):
    def __init__(self, fileName, tagName:str="serve", allTags:"list[str]"=("test", "notest", "donttest", "thumbnail", "export", "serve", "noserve", "dash")):
        """Grabs source code from a Jupyter notebook. Will grab cells with the comment
like ``# serve`` in the first line.

See :meth:`tag_serve` to see more about its options

:param fileName: notebook path
:param tagName: which tag to extract out?
:param allTags: all possible custom tags that you use. It might complain if there's a tag in your
    notebook that it doesn't know about, so add all of your commonly used tags here"""
        super().__init__(); self.fileName = fileName; self.tagName = tagName; self.allTags = allTags
    def fetchSource(self):
        a = cli.nb.cells(self.fileName) | cli.filt(cli.op()["cell_type"] == "code") | cli.aS(list); self.l["sourceType"] = "notebook"
        self.l["sourceCode"] = a | cli.nb.pretty(whitelist=[self.tagName]) | (cli.op()["source"] | ~cli.head(1)).all() | cli.joinStreams() | cli.deref()
        self.l["tags"] = a | cli.op()["source"].all() | cli.filt("x") | cli.item().all() | cli.filt(cli.op().startswith("#")) | cli.deref()
        # figures out build vars here, like node & condaEnv
        self.l["node"] = None; self.l["condaEnv"] = None; self.l["condaPath"] = "~/miniconda3"
        def serve(node:str=None, condaEnv:str=None, condaPath:str="~/miniconda3"): self.l["node"] = self.l["node"] or node; self.l["condaEnv"] = self.l["condaEnv"] or condaEnv; self.l["condaPath"] = self.l["condaPath"] or condaPath
        for tag in self.l["tags"]: nb.executeTags(tag, defaultdict(lambda: 0, {**{x:0 for x in self.allTags}, "serve": serve}))
class FromPythonFile(k1.Callback):
    def __init__(self, fileName):
        """Grabs source code from a python file."""
        super().__init__(); self.fileName = fileName
    def fetchSource(self): self.l["sourceCode"] = cli.cat(self.fileName) | cli.deref(); self.l["sourceType"] = "file"
class BuildPythonFile(k1.Callback):
    def __init__(self, port=None):
        """Builds the output Python file, ready to be served on localhost.

:param port: which port to run on localhost. If not given, then a port will
    be picked at random, and will be available at ``cbs.l['port']``"""
        super().__init__(); self.port = port; self.suffix = "suffix.py"
    def buildPythonFile(self):
        self.l["pythonFile"] = ["from k1lib.imports import *", *self.l["sourceCode"]] | cli.file(); port = self.port
        self.l["metaFile"] = metaFile = "" | cli.file(); os.remove(metaFile) # grabs temp meta file for communication, on localhost, not remote
        if self.l["node"] is None: # grabs random free port if one is not available
            if port is None: import socket; sock = socket.socket(); sock.bind(('', 0)); port = sock.getsockname()[1]; sock.close()
        else:
            kw = {"node": self.l["node"], "condaEnv": self.l["condaEnv"], "condaPath": self.l["condaPath"]}
            if port is None: port = execPy('import socket; sock = socket.socket(); sock.bind(("", 0)); port = sock.getsockname()[1]; sock.close(); print(port)', **kw)[0]
        # actually has enough info to build the final file
        self.l["port"] = port; node = self.l["node"]; (cli.cat(f"{basePath}{self.suffix}") | cli.op().replace("SOCKET_PORT", f"{port}").replace("META_FILE", metaFile).all()) >> cli.file(self.l["pythonFile"])
        if node: fn = self.l["pythonFile"]; None | cli.cmd(f"scp {fn} {node}:{fn}") | cli.deref()
class BuildDashFile(BuildPythonFile):
    def __init__(self):
        """Builds the output Python file for a Dash app, ready to be served on localhost"""
        super().__init__()
        self.suffix = "suffix-dash.py"
class StartServer(k1.Callback):
    def __init__(self, maxInitTime=10):
        """Starts the server, verify that it starts okay and dumps meta information (including
function signatures) to ``cbs.l``

:param maxInitTime: time to wait in seconds until the server is online before declaring it's unsuccessful"""
        super().__init__(); self.maxInitTime = maxInitTime
    def startServer(self):
        pythonFile = self.l["pythonFile"]; metaFile = self.l["metaFile"]; port = self.l["port"]; maxInitTime = self.maxInitTime
        node = self.l["node"]; condaEnv = self.l["condaEnv"]; condaPath = self.l["condaPath"]; startTime = time.time()
        # print(f"{pythonFile=} {metaFile=} {port=} {maxInitTime=} {node=} {condaEnv=} {condaPath=}")
        if node is None:
            None | cli.cmd(f"python {pythonFile} &"); count = 0
            while not os.path.exists(metaFile):
                if time.time()-startTime > maxInitTime: raise Exception(f"Tried to start server up, but no responses yet. Port: {port}, pythonFile: {pythonFile}, metaFile: {metaFile}")
                time.sleep(0.1)
            self.l["meta"] = meta = metaFile | cli.cat(text=False) | cli.aS(dill.loads)
        else:
            if condaEnv: None | cli.cmd(f"ssh {node} 'source {condaPath}/bin/activate {condaEnv} && nohup python {pythonFile}' &");
            else: None | cli.cmd(f"ssh {node} 'nohup python {pythonFile}' &");
            while not int(None | cli.cmd(f"ssh {node} 'if [ -e {metaFile} ]; then echo 1; else echo 0; fi'") | cli.item()):
                if time.time()-startTime > maxInitTime: raise Exception(f"Tried to start server up, but no responses yet. Port: {port}, pythonFile: {pythonFile}, metaFile: {metaFile}")
                time.sleep(0.5)
            self.l["meta"] = meta = dill.loads(b"".join(None | cli.cmd(f"ssh {node} 'cat {metaFile}'", text=False)))
class GenerateHtml(k1.Callback):
    def __init__(self, serverPrefix=None, htmlFile=None, title="Interactive demo"):
        """Generates a html file that communicates with the server.

:param serverPrefix: prefix of server for back and forth requests, like "https://example.com/proj1". If
    empty, tries to grab ``cbs.l["serverPrefix"]``, which you can deposit from your own callback. If
    that's not available then it will fallback to ``localhost:port``
:param htmlFile: path of the target html file. If not specified then a temporary file
    will be created and made available in ``cbs.l["htmlFile"]``
:param title: title of html page"""
        super().__init__(); self.serverPrefix = serverPrefix; self.htmlFile = htmlFile; self.title = title
    def generateHtml(self):
        meta = dict(self.l["meta"])
        meta["fetch"]["js"] = meta["fetch"]["js"].replace("PREFIX", self.l["prefix"] or "PREFIX_")
        meta["fetch"]["py"] = meta["fetch"]["py"].replace("PREFIX", self.l["prefix"] or "PREFIX_")
        meta["fetch"]["k1"] = meta["fetch"]["k1"].replace("PREFIX", self.l["prefix"] or "PREFIX_")
        replaceNewlineWithBr = op().split("<!-- k1lib.raw.start -->") | apply(op().split("<!-- k1lib.raw.end -->")) | head(1).split() | (op().replace("\n", "<br>").all(2)) + apply(op().replace("\n", "<br>"), 1) | joinSt(2) | join("")
        replaces = cli.op().replace("META_JSON", base64.b64encode(_json.dumps(meta).encode()).decode())\
            .replace("SERVER_PREFIX", self.serverPrefix or self.l["serverPrefix"] or f"http://localhost:{self.l['port']}")\
            .replace("TITLE", self.title).replace("INTRO", meta["mainDoc"] | replaceNewlineWithBr | op().replace("\ue157", "\n"))\
            .replace("SOURCE_CODE", pretty_py(meta["source"]))
        self.l["htmlFile"] = cli.cat(f"{basePath}main.html") | replaces.all() | cli.file(self.htmlFile)
def commonCbs():
    """Grabs common callbacks, including :class:`BuildPythonFile` and :class:`StartServer`"""
    return k1.Callbacks().add(BuildPythonFile()).add(StartServer());
def serve(cbs):
    """Runs the serving pipeline."""
    import flask, flask_cors
    cbs.l = defaultdict(lambda: None)
    cbs("begin")
    cbs("fetchSource") # fetches cells
    cbs("beforeBuildPythonFile"); cbs("buildPythonFile") # builds python server file
    cbs("beforeStartServer"); cbs("startServer") # starts serving the model on localhost and add more meta info
    cbs("beforeGenerateHtml"); cbs("generateHtml") # produces a standalone html file that provides interactive functionalities
    cbs("end")
    return cbs

In [18]:
#export
def cutoffLongText(s):
    if len(s) > 50:
        if isinstance(s, str):   return s[:50]+ "..."+s[-10:]
        if isinstance(s, bytes): return s[:50]+b"..."+s[-10:]
    return s
def fetch_js(meta):
    a = meta["annos"].items() | filt(op()!="return", 0) | ~apply(lambda x,y: [x,y,meta["defaults"][x]]) | deref()
    ans = []; ret = meta["annos"]["return"]
    for arg, anno, default in a:
        if anno in ("int", "float", "slider", "date", "text"): ans.append(f"    '{arg}': {_json.dumps(default[0])}")
        elif anno in ("apiKey",):     ans.append(f"    '{arg}': '<api key here>'")
        elif anno in ("checkbox",):   ans.append(f"    '{arg}': {_json.dumps(default)}")
        elif anno in ("json",):       ans.append(f"    '{arg}': {_json.dumps(default[0])} /* raw json object, no need to do JSON.stringify() */")
        elif anno in ("bytes",):      ans.append(f"    '{arg}': {cutoffLongText(_json.dumps(default))} /* base64-encoded bytes */")
        elif anno in ("image",):      ans.append(f"    '{arg}': {cutoffLongText(_json.dumps(default))} /* base64-encoded bytes of image in popular formats like jpg, png */")
        elif anno in ("serialized",): ans.append(f"    '{arg}': {cutoffLongText(_json.dumps(default))} /* base64-encoded bytes, encoded by python module `dill` */")
        elif anno in ("dropdown",):   ans.append(f"    '{arg}': {_json.dumps(default[1][default[0]])}")
    b = ",\n".join(ans)
    s = f"""
let result = (await (await fetch("https://local.mlexps.com/routeServer/PREFIX", {{
  method: "POST",
  body: JSON.stringify({{
{b}
  }}),
  headers: {{
    "Content-Type": "application/json",
  }}
}})).json()).data;"""
    if ret in ("int", "float"): s = f"{s} // returns {ret}"
    elif ret in ("text", "html"): s = f"{s} // returns string"
    elif ret in ("json",): s = f"{s} // returns js object, no need to do JSON.parse()"
    elif ret in ("bytes", "serialized"): s = f"{s} // returns base64-encoded bytes"
    elif ret in ("image",): s = f"{s} // returns base64-encoded bytes of an image, typically in jpg"
    return s
def fetch_py(meta):
    a = meta["annos"].items() | filt(op()!="return", 0) | ~apply(lambda x,y: [x,y,meta["defaults"][x]]) | deref()
    ans = []; ret = meta["annos"]["return"]
    for arg, anno, default in a:
        if anno in ("int", "float", "slider", "date", "text"): ans.append(f"    '{arg}': {_json.dumps(default[0])},")
        elif anno in ("apiKey",):     ans.append(f"    '{arg}': '<api key here>',")
        elif anno in ("checkbox",):   ans.append(f"    '{arg}': {default},")
        elif anno in ("json",):       ans.append(f"    '{arg}': {default[0]}, # raw python object, no need to do json.dumps()")
        elif anno in ("bytes",):      ans.append(f"    '{arg}': base64.b64encode({cutoffLongText(base64.b64decode(default))}).decode(), # base64-encoded bytes")
        elif anno in ("image",):      ans.append(f"    '{arg}': base64.b64encode({cutoffLongText(base64.b64decode(default))}).decode(), # base64-encoded bytes of image in popular formats like jpg, png")
        elif anno in ("serialized",): ans.append(f"    '{arg}': base64.b64encode(dill.dumps(<raw_python_object_here>)).decode(), # base64-encoded bytes, encoded by python module `dill`")
        elif anno in ("dropdown",):   ans.append(f"    '{arg}': {_json.dumps(default[1][default[0]])},")
    b = "\n".join(ans)
    s = f"""
import requests, base64, dill, json
result = requests.post("https://local.mlexps.com/routeServer/PREFIX", json={{
{b}
}}).json()["data"]"""
    if ret in ("int", "float"): s = f"{s} # returns {ret}"
    elif ret in ("text", "html"): s = f"{s} # returns string"
    elif ret in ("json",): s = f"{s} # returns js object, no need to do json.loads()"
    elif ret in ("bytes",): s = f"{s} # returns base64-encoded bytes\nresult = base64.b64decode(result.encode())"
    elif ret in ("serialized",): s = f"{s} # returns base64-encoded bytes\nresult = dill.loads(base64.b64decode(result.encode()))"
    elif ret in ("image",): s = f"{s} # returns base64-encoded bytes of an image, typically in jpg"
    return s
def fetch_k1(meta):
    a = meta["annos"].items() | filt(op()!="return", 0) | ~apply(lambda x,y: [x,y,meta["defaults"][x]]) | deref()
    ans = []; ret = meta["annos"]["return"]
    for arg, anno, default in a:
        if anno in ("int", "float", "slider", "date", "text"): ans.append(f"    '{arg}': {_json.dumps(default[0])},")
        elif anno in ("apiKey",):     ans.append(f"    '{arg}': '<api key here>',")
        elif anno in ("checkbox",):   ans.append(f"    '{arg}': {default},")
        elif anno in ("json",):       ans.append(f"    '{arg}': {default[0]},")
        elif anno in ("bytes",):      ans.append(f"    '{arg}': {cutoffLongText(base64.b64decode(default))},")
        elif anno in ("image",):      ans.append(f"    '{arg}': 'some/image/path.jpg' | toImg(), # raw PIL image")
        elif anno in ("serialized",): ans.append(f"    '{arg}': <raw_python_object_here>,")
        elif anno in ("dropdown",):   ans.append(f"    '{arg}': {_json.dumps(default[1][default[0]])},")
    b = "\n".join(ans)
    s = f"""{{\n{b}\n}} | kapi.demo("PREFIX")"""
    if ret in ("int", "float"): s = f"{s} # returns {ret}"
    elif ret in ("text", "html"): s = f"{s} # returns string"
    elif ret in ("json",): s = f"{s} # returns js object, no need to do json.loads()"
    elif ret in ("bytes",): s = f"{s} # returns raw bytes, no need to base64-decode"
    elif ret in ("serialized",): s = f"{s} # returns raw Python object, no need to base64-decode and dill-decode"
    elif ret in ("image",): s = f"{s} # returns PIL image, no need to base64-decode"
    return s

In [7]:
#export
class baseType:
    def __init__(self):
        """Base type for all widget types"""
        pass
    def getConfig(self): return NotImplemented
class text(baseType):
    def __init__(self, multiline:bool=True, password:bool=False):
        """Represents text, either on single or multiple lines.
If `password` is true, then will set multiline to false automatically,
and creates a text box that blurs out the contents. Example::

    def endpoint(s:serve.text()="abc") -> str: pass

For inputs only. Use ``str`` for outputs"""
        super().__init__(); self.multiline = multiline if not password else False; self.password = password
    def __repr__(self): return f"<text multiline={self.multiline}>"
class slider(baseType):
    def __init__(self, start:float, stop:float, intervals:int=100):
        """Represents a slider from `start` to `stop` with a bunch of
intervals in the middle. If `defValue` is not specified, uses the
middle point between start and stop. Example::

    def endpoint(a:serve.slider(2, 3.2)=2.3) -> str: pass

For inputs only"""
        super().__init__(); self.start = start; self.stop = stop; self.intervals = intervals; self.dt = (stop-start)/intervals; self.step = self.dt
    def __repr__(self): return f"<slider {self.start}---{self.intervals}-->{self.stop}>"
class html(baseType):
    def __init__(self):
        """Raw html.
Example::

    def endpoint() -> serve.html(): pass

For outputs only"""
        super().__init__()
    def __repr__(self): return f"<html>"
class json(baseType):
    def __init__(self):
        """Raw json.
Example::

    def endpoint(a:serve.json()={"a": 3}) -> serve.json(): pass

For inputs and outputs"""
        super().__init__()
    def __repr__(self): return f"<json>"
class date(baseType):
    def __init__(self, min=None, max=None):
        """Local date time (no timezone information).
Example::

    def endpoint(d:serve.date()="2023-12-07T00:00:00") -> str: pass

:param min: min date, also in format '2023-12-07T00:00:00'"""
        super().__init__(); self.minDate = min; self.maxDate = max
    def __repr__(self): return f"<date>"
class serialized(baseType):
    def __init__(self):
        """For serialized objects using :mod:`dill`.
Example::

    def endpoint(a:serve.serialized()) -> serve.serialized():
        return {"any": "data structure", "you": "want", "even": np.random.randn(100)}
"""
        super().__init__()
    def __repr__(self): return f"<serialized>"
class apiKey(baseType):
    def __init__(self, apiKey=str):
        """Protects your endpoint with an api key.
Example::

    def endpoint(apiKey:serve.apiKey("your api key here")="") -> str: pass

When compiled, your api key won't appear anywhere, not in the html, not in the meta
files, and someone calling the endpoint must specify it, else it will just errors out"""
        super().__init__(); self.apiKey = apiKey
    def __repr__(self): return f"<apiKey>"
def refine(param:str, anno:baseType, default): # anno is not necessarily baseType, can be other types like "int"
    if anno == int: return [param, "int", [default, False], None]
    if anno == float: return [param, "float", [default, False], None]
    multiline = lambda s: len(s.split("\n")) > 1 or len(s) > 100
    if anno == bool: return [param, "checkbox", default, None]
    if anno == str: return [param, "text", [default, multiline(default or "")], None]
    if isinstance(anno, text): return [param, "text", [default, anno.multiline, anno.password], None]
    if isinstance(anno, slider): return [param, "slider", [default, anno.start, anno.stop, anno.dt], None]
    if isinstance(anno, range): return [param, "slider", [default, anno.start, anno.stop, anno.step], None]
    byte2Str = aS(base64.b64encode) | op().decode("ascii")
    if hasPIL and anno == PIL.Image.Image: return [param, "image", (default | toBytes() | byte2Str) if default is not None else None, None]
    if anno == bytes: return [param, "bytes", (default | byte2Str) if default is not None else None, None]
    if isinstance(anno, serialized): return [param, "serialized", (default | aS(dill.dumps) | byte2Str) if default is not None else None, None]
    if isinstance(anno, list): anno | apply(str) | deref(); return [param, "dropdown", [anno.index(default), anno], None]
    if isinstance(anno, html): return [param, "html", [default], None]
    if isinstance(anno, json): return [param, "json", [default, True], None]
    if isinstance(anno, date): return [param, "date", [default, anno.minDate, anno.maxDate], None]
    if isinstance(anno, apiKey): return [param, "apiKey", [default, False, True], anno.apiKey]
    raise Exception(f"Unknown type {anno}")
def analyze(f):
    spec = getattr(f, "fullargspec", inspect.getfullargspec(f)); args = spec.args; n = len(args)
    annos = spec.annotations; defaults = spec.defaults or ()
    docs = (f.__doc__ or "").split("\n") | grep(":param", sep=True).till() | filt(op().ab_len() > 0) | op().strip().all(2) | (join(" ") | op().split(":") | ~aS(lambda x, y, *z: [y.split(" ")[1], ":".join(z).strip()])).all() | toDict()
    mainDoc = (f.__doc__ or " ").split("\n") | grep(".", sep=True).till(":param") | breakIf(op()[0].startswith(":param")) | join("\n").all() | join("\n")

    if len(annos) != n + 1: raise Exception(f"Please annotate all of your arguments ({n} args + 1 return != {len(annos)} annos). Args: {args}, annos: {annos}")
    if len(defaults) != n: raise Exception(f"Please specify default values for all of your arguments ({n} args != {len(defaults)} default values)")
    a = [args, args | lookup(annos), defaults] | transpose() | ~apply(refine) | deref()
    ret = refine("return", annos["return"], None)[1]; defaults = a | cut(0, 2) | toDict()
    annos = a | cut(0, 1) | toDict(); annos["return"] = ret; privates = a | cut(0, 3) | toDict()
    if ret == "slider": raise Exception(f"Return value is a slider, which doesn't really make sense")
    if ret == "date": raise Exception(f"Return value is a date, which doesn't really make sense. Use `str` instead")
    if ret == "apiKey": raise Exception(f"Return value is an api key, which doesn't really make sense")
    if ret == "checkbox": raise Exception(f"Return value is a checkbox, which doesn't really make sense. Use `serve.json()` instead")
    for anno in [v for k,v in annos.items() if k != "return"]:
        if anno == "html": raise Exception("One of the input arguments is serve.html(), which doesn't really make sense. Use serve.text() instead")

    # args:list, annos:dict, defaults:list, docs:dict

    meta = {"args": args, "annos": annos, "defaults": defaults, "privates": privates, "docs": docs,
     "mainDoc": mainDoc, "source": inspect.getsource(f), "pid": os.getpid()}
    f_js = pretty_js(fetch_js(meta)); f_py = pretty_py(fetch_py(meta)); f_k1 = pretty_py(fetch_k1(meta))
    return {**meta, "fetch": {"js": f_js, "py": f_py, "k1": f_k1}}
def webToPy(o, klass:baseType):
    if klass == "int": return int(o)
    if klass in ("float", "text", "checkbox", "dropdown", "slider", "apiKey", "date", "json"): return o
    o = str(o)
    if klass == "bytes": return base64.b64decode(o)
    if klass == "serialized": return dill.loads(base64.b64decode(o))
    if klass == "image": return o | aS(base64.b64decode) | toImg()
    if klass == "html": return k1lib.viz.Html(base64.b64decode(o).decode())
    return NotImplemented
def pyToWeb(o, klass:baseType):
    if klass in ("int", "float", "text", "checkbox", "slider", "apiKey", "date", "json"): return o
    if klass == "bytes": return base64.b64encode(o).decode()
    if klass == "serialized": return base64.b64encode(dill.dumps(o)).decode()
    if klass == "image": return base64.b64encode(o | toBytes()).decode()
    if klass == "dropdown": return o;
    if klass == "html": return o.encode() | aS(base64.b64encode) | op().decode()
    return NotImplemented

In [4]:
import PIL.Image
def f(a:bool=True, b:float=5, c:text(True)="x^3", d:range(5)=3, e:slider(5, 6)=5.5, f:["a", "b"]="b") -> PIL.Image.Image:
    pass
analyze(f)

{'args': ['a', 'b', 'c', 'd', 'e', 'f'],
 'annos': {'a': 'checkbox',
  'b': 'float',
  'c': 'text',
  'd': 'slider',
  'e': 'slider',
  'f': 'dropdown',
  'return': 'image'},
 'defaults': {'a': True,
  'b': [5, False],
  'c': ['x^3', True, False],
  'd': [3, 0, 5, 1],
  'e': [5.5, 5, 6, 0.01],
  'f': [1, ['a', 'b']]},
 'docs': {},
 'mainDoc': ' ',
 'source': 'def f(a:bool=True, b:float=5, c:text(True)="x^3", d:range(5)=3, e:slider(5, 6)=5.5, f:["a", "b"]="b") -> PIL.Image.Image:\n    pass\n',
 'pid': 3172769}

In [5]:
def f(a:bool=True) -> html():
    pass
analyze(f)

{'args': ['a'],
 'annos': {'a': 'checkbox', 'return': 'html'},
 'defaults': {'a': True},
 'docs': {},
 'mainDoc': ' ',
 'source': 'def f(a:bool=True) -> html():\n    pass\n',
 'pid': 3172769}

In [6]:
def f(a:bool=True) -> json():
    pass
analyze(f)

{'args': ['a'],
 'annos': {'a': 'checkbox', 'return': 'json'},
 'defaults': {'a': True},
 'docs': {},
 'mainDoc': ' ',
 'source': 'def f(a:bool=True) -> json():\n    pass\n',
 'pid': 3172769}

In [1]:
!../../export.py serve/main --upload=True

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-02-01 16:27:35,209	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-02-01 16:27:35,216	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- exportAll
14186   0   60%   
9318    1   40%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5.2
Uninstalling k1lib-1.5.2:
  Successfully uninstalled k1lib-1.5.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and o

In [5]:
!../../export.py serve/main

2024-03-08 05:04:38,999	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-03-08 05:04:39,009	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15666   0   61%   
10027   1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.6
Uninstalling k1lib-1.6:
  Successfully uninstalled k1lib-1.6
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg

In [89]:
!../../export.py serve/main --bootstrap=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-01-04 06:24:51,282	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-01-04 06:24:51,297	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- bootstrapping
Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/serve/../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards